In [1]:
from time import time
import pandas as pd
import numpy as np
import SARpy
import operator
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from sklearn.model_selection import train_test_split
from pprint import pprint
from rdkit.Chem import AllChem as Chem
from matplotlib import pyplot as plt
import matplotlib
import matplotlib.patches as mpatches
import scipy.stats as stats
import statsmodels.stats.multitest as smm

SyntaxError: Missing parentheses in call to 'print'. Did you mean print('\n %s structures matched' % c)? (SARpy.py, line 249)

In [ ]:
structures = pd.read_csv("./Most_No_DILIConcern_Dataset_for_SAs.csv")
structures

In [ ]:
Compound_Name = structures['Compound Name'].tolist()
SMILES = structures.loc[:,['SMILES']]

In [ ]:
def label (x):
    if x == 'vMost-DILI-Concern':
        return 1
    if x == 'vNo-DILI-Concern':
        return 0

In [ ]:
SMILES['DILI_LABEL'] = structures['vDILIConcern'].apply(label)
SMILES.to_csv("Data_for_SARpy.csv")

In [ ]:
data = pd.read_csv("Data_for_SARpy.csv")
x = data[['SMILES','DILI_LABEL']]
y = data['DILI_LABEL']
x.to_csv("Train_Data_for_SARpy.csv")

In [ ]:
x.head()

In [ ]:
filt1 = SARpy.Filter('DILI_LABEL',0, operator.eq)
filt2 = SARpy.Filter('DILI_LABEL',1, operator.eq)
dictionary = {'INACTIVE':filt1, 'ACTIVE':filt2}
dataset = SARpy.loadDataset("Train_Data_for_SARpy.csv",'csv',dictionary,'SMILES')

In [ ]:
SARpy.fragmentize(dataset,2,15)
rules = SARpy.extract(dataset, minHits = 5,minLR = 1, minPrecision = 0) # 'OPTIMAL'
SARpy.saveSmarts(rules,'ruleset.txt')

In [ ]:
## Way to get SARpy substructures for DILI (not for noDILI)

accuracy_list = []
sensitivity_list = []
specificity_list = []
SMARTS_list = []
LR_list = []
Target_list = []
test_comp_matched_list = []
for rule in rules:
    myrules = [rule,rule]
    SARpy.saveSmarts(myrules,'ruleset.txt')
    alerts = pd.read_csv("ruleset.txt", sep='\t')
    alerts = alerts.ix[1]
    #print(alerts)
    SMARTS = alerts['SMARTS']
    TARGET = alerts['Target']
    Target_list.append(TARGET)
#    if TARGET == 'ACTIVE':
        
    
    LR = alerts['Training LR']
    pred = SARpy.predict(myrules,dataset) #dataset_test
    if pred >= 1:
        accuracy,sensitivity,specificity = SARpy.validate(dataset) #
            #print(accuracy,sensitivity,specificity,LR, SMARTS)
        pred = (pred/167.0)*100
        test_comp_matched_list.append((pred))
        accuracy_list.append(accuracy)
        sensitivity_list.append(sensitivity)
        specificity_list.append(specificity)
        SMARTS_list.append(SMARTS)
        LR_list.append(LR)
            
            
        
    else:
        continue
        
        
df = pd.DataFrame()
df['SMARTS'] = SMARTS_list
df['TYPE'] = Target_list
df['Accuracy'] = accuracy_list
df['Sensitivity'] = sensitivity_list
df['Specificity'] = specificity_list
df['LR_training'] = LR_list
df

In [ ]:
df['Source'] = 'SARpy'

In [ ]:
# Add MOSS substructures (from KNIME workflow)
MOSS_SS_Active = pd.read_csv("./moss_results.csv")

In [ ]:
MOSS_SS_Active['Fragment'].tolist()

In [ ]:

MOSS_SS_Active = MOSS_SS_Active['Fragment'].tolist()

MOSS_SS_Active = dict(zip(MOSS_SS_Active, ['ACTIVE'] * len(MOSS_SS_Active)))


MOSS_SS_Active = pd.DataFrame(MOSS_SS_Active.items(), columns=['SMARTS', 'TYPE'])


In [ ]:
#MOSS_SS = pd.concat([MOSS_SS_Active,MOSS_SS_Inactive])
MOSS_SS = MOSS_SS_Active
MOSS_SS['Source'] = 'MOSS'
df = pd.concat([MOSS_SS,df])

In [ ]:
df

In [ ]:
# Add literature SMARTS

Literature_SMARTS = pd.read_csv("./Literature_SMARTS.csv")

In [ ]:
Literature_SMARTS['Source'] = 'Hewitt et al. (2013)'
Literature_SMARTS['Source'][16:] = "Liu et al. (2015)"
Literature_SMARTS

In [ ]:
df = pd.concat([Literature_SMARTS,df])

In [ ]:
df.reset_index(inplace=True)

In [ ]:
x

In [ ]:
## Tabulate results (throughout this script PPV is used as a synomym for precision)



df2 = pd.DataFrame(index = x['SMILES'])
df2['DILI_LABEL'] = x['DILI_LABEL'].to_list()
df2['Compound_Name'] = Compound_Name

substructure_stats = pd.DataFrame()

#generate FPs
smis = list(df2.index)

mols = [Chem.MolFromSmiles(smile) for smile in smis]

fps_bit = [list(Chem.GetMorganFingerprintAsBitVect(mol,2, nBits=2048)) for mol in mols]


h = 0 # count TPs
l = 0 # count FPs
smart_list = []
PPV_list = []
percent_hits = []
labels = []
sources = []
odds_ratio_list = []
p_value_list = []
perc_hit_list = []
type_list = []
DB_count_list = []
for index, row in df.iterrows():
    i=0
    j=0
    k=0
    z=0
    t=0
    c=0
    v=0
    DB_count = 0
    hit_list = []
    smart = row['SMARTS']
    TYPE = row['TYPE']
    source = row['Source']
    
 
    if TYPE == 'ACTIVE':
#        for DB in Drugbank_comps:
#            m = Chem.MolFromSmiles(DB)
#            patt = Chem.MolFromSmarts(smart)
#            if m.HasSubstructMatch(patt):
#                DB_count+=1
#            else:
#                continue
#        DB_count_list.append(DB_count)
    
    
        for index, row in x.iterrows():
            label = row['DILI_LABEL']
            a = row['SMILES']   
            m = Chem.MolFromSmiles(a)
            patt = Chem.MolFromSmarts(smart)
            if m.HasSubstructMatch(patt) is True:
                i+=1
            if m.HasSubstructMatch(patt) is True and label == 1: # True positive
                #print('TP')
                j+=1
            if m.HasSubstructMatch(patt) is True and label == 0: # False positive
                #print('FP')
                k+=1
            if m.HasSubstructMatch(patt) is True:
                hit_list.append(1)
            if m.HasSubstructMatch(patt) is False:
                hit_list.append('0')
                
                
            if m.HasSubstructMatch(patt) is True and label == 1: # compound is DILI and has SS == TP
                z+=1
            if m.HasSubstructMatch(patt) is False and label == 1: # compound is DILI and does not have SS == FN
                t+=1
            if m.HasSubstructMatch(patt) is True and label == 0: # compound is noDILI and has SS == FP
                c+=1
            if m.HasSubstructMatch(patt) is False and label == 0: # compound is noDILI and does not have SS == TN
                v+=1    
                
        #if source == 'Hewitt et al. (2013)':
           # sources.append(0)
        if source == 'Liu et al. (2015)':
            sources.append(1)
        elif source == 'SARpy':
            sources.append(2)
        elif source == 'MOSS':
            sources.append(3)
        else:
            continue
                
            
        #print(j)
        #print(len(x.loc[x['DILI_LABEL']==1]))
        perc_hits = (j*1.0/len(x.loc[x['DILI_LABEL']==1]))*100.0
        percent_hits.append(perc_hits)
        labels.append(0)

    
        if ((j+k) == 0):
            PPV = 0
        else:

            PPV = (j/(j+k*1.0))
            print('{0:.3f}'.format(PPV), smart, perc_hits) # positive predictive value (most important for substructures to classify DILI)


        #PPV_other = z/(z+c)
        print(PPV,j,z,k,c)

        PPV_list.append(PPV)

        # Fisher one-sided test

        oddsratio, pvalue = stats.fisher_exact([[z, c], [t, v]], alternative='greater')

        odds_ratio_list.append(oddsratio)
        p_value_list.append(pvalue)
        perc_hit_list.append(perc_hits)
        type_list.append(TYPE)
        smart_list.append(smart)


        if PPV >= 0.0: # filter for PPV

            #df2[smart+'_'+TYPE] = hit_list
            print(TYPE,i,j,k,pvalue,oddsratio,source,len(x['SMILES']),len(x.loc[x['DILI_LABEL']==1]),len(x.loc[x['DILI_LABEL']==0]))
            #df2[smart] = hit_list

            h+=j
            l+=k
            #PPV_list[smart] = PPV
        else:
            continue
        
substructure_stats = pd.DataFrame(index = smart_list)
substructure_stats['PPV'] = PPV_list
substructure_stats['Source'] = sources
substructure_stats['odds_ratio'] = odds_ratio_list
substructure_stats['p_value'] = p_value_list
substructure_stats['perc_hits'] = perc_hit_list
substructure_stats['type'] = type_list


print(h,l) # number of TPs and FPs

In [ ]:
print(len(smart_list),len(PPV_list),len(sources))

In [ ]:
substructure_stats.loc[substructure_stats['Source']==3]['PPV'].describe()

In [ ]:
len(type_list)

In [ ]:
rej, pval_corr = smm.multipletests(p_value_list, method='fdr_bh')[:2]
substructure_stats['BH_Adjusted_p_value'] = list(pval_corr)

In [ ]:
substructure_stats

In [ ]:
substructure_stats.to_csv("./Structural_alerts.csv")